In [112]:
import urllib.request
import pandas as pd
import numpy as np


In [113]:

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", 
                           filename="./data/ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", 
                           filename="./data/ratings_test.txt")

('./data/ratings_test.txt', <http.client.HTTPMessage at 0x1d40ddd2b80>)

In [114]:
train_data=pd.read_table('./data/ratings_train.txt')
test_data=pd.read_table('./data/ratings_test.txt')


In [115]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())
print(len(train_data),len(test_data))

id          0
document    5
label       0
dtype: int64
id          0
document    3
label       0
dtype: int64
150000 50000


In [116]:
import re
re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]','',sample_hangul)

'저재ㅓ재버장배업ㅈㄹㅓㄹ'

In [117]:
#한글을 제외한 문자를 ''로 변경
train_data['document']=train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ' ']','',regex=True)
test_data['document']=test_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ' ']','',regex=True)

#''를 np.nan으로 변경 ->nan 을 제거
train_data['document'].replace('',np.nan,inplace=True)
train_data.dropna(inplace=True)
test_data['document'].replace('',np.nan,inplace=True)
test_data.dropna(inplace=True)

print(len(train_data),len(test_data))
print(train_data['document'])

149186 49726
0                                         아 더빙 진짜 짜증나네요 목소리
1                                흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                 교도소 이야기구먼 솔직히 재미는 없다평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
                                ...                        
149995                                      인간이 문제지 소는 뭔죄인가
149996                                           평점이 너무 낮아서
149997                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998                           청춘 영화의 최고봉방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149186, dtype: object


In [118]:
# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정합니다.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import numpy as np

x_train = train_data['document'].to_list()
y_train= train_data['label'].to_numpy()


token=Tokenizer()
token.fit_on_texts(x_train)
x=token.texts_to_sequences(x_train)
padded_x=pad_sequences(x, maxlen=10)


In [1]:
word_size=len(token.word_index)+1
word_size

NameError: name 'token' is not defined

In [120]:

model=Sequential()
model.add(Embedding(word_size,8,input_length=10))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 10, 8)             2396832   
                                                                 
 flatten_15 (Flatten)        (None, 80)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 81        
                                                                 
Total params: 2,396,913
Trainable params: 2,396,913
Non-trainable params: 0
_________________________________________________________________


In [121]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_x, y_train, epochs=20,batch_size=512)

          

Epoch 1/20
292/292 [==============================] - 9s 28ms/step - loss: 0.6352 - accuracy: 0.6977
Epoch 2/20
292/292 [==============================] - 8s 27ms/step - loss: 0.4423 - accuracy: 0.8481
Epoch 3/20
292/292 [==============================] - 8s 27ms/step - loss: 0.3143 - accuracy: 0.9103
Epoch 4/20
292/292 [==============================] - 8s 28ms/step - loss: 0.2282 - accuracy: 0.9423
Epoch 5/20
292/292 [==============================] - 8s 27ms/step - loss: 0.1679 - accuracy: 0.9601
Epoch 6/20
292/292 [==============================] - 8s 28ms/step - loss: 0.1264 - accuracy: 0.9716
Epoch 7/20
292/292 [==============================] - 8s 29ms/step - loss: 0.0977 - accuracy: 0.9778
Epoch 8/20
292/292 [==============================] - 9s 31ms/step - loss: 0.0776 - accuracy: 0.9829
Epoch 9/20
292/292 [==============================] - 9s 30ms/step - loss: 0.0633 - accuracy: 0.9861
Epoch 10/20
292/292 [==============================] - 8s 28ms/step - loss: 0.0529 - accura

In [124]:
x_test = test_data['document'].to_list()
y_test= test_data['label'].to_numpy()


token=Tokenizer()
token.fit_on_texts(x_test)
y=token.texts_to_sequences(x_test)
padded_y=pad_sequences(y, maxlen=10)
word_size=len(token.word_index)+1



In [125]:
model.evaluate(padded_y, y_test)[1]

1554/1554 [==============================] - 2s 1ms/step - loss: 2.0218 - accuracy: 0.5041


0.504142701625824